In [1]:
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqUtils import seq3

In [2]:
mut_df = pd.read_csv('../data/mutation_dists.filtered.csv')

In [3]:
#pos here starts with zero, I want to start it with 1
mut_df['pos']=mut_df['pos']+1

In [4]:
ref_df = pd.read_csv('../data/U_ideal_table.csv', index_col=0)

C:\Users\voron\AppData\Local\Temp\ipykernel_21508\194572541.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  ref_df = pd.read_csv('../data/U_ideal_table.csv', index_col=0)


In [5]:
ref_df.head()

,Pos,RefNuc,GenName,GenType,CodonNumber,RefCodon,RefAa,NucInCodon,AltNuc,AltCodon,AltAa,AaSub,NeighL,NeighR,MutExp,MutObsNC,IsStem,SsPairs,Quadr,Strand_Quadr
0,1,A,5UTR,untranslated,NaN,NaN,NaN,None,U,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN
1,1,A,5UTR,untranslated,NaN,NaN,NaN,None,G,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN
2,1,A,5UTR,untranslated,NaN,NaN,NaN,None,C,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN
3,2,U,5UTR,untranslated,NaN,NaN,NaN,None,A,NaN,NaN,NaN,A,U,NaN,NaN,NaN,NaN,NaN,NaN
4,2,U,5UTR,untranslated,NaN,NaN,NaN,None,G,NaN,NaN,NaN,A,U,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
merge_df = ref_df[['Pos', 'GenName','CodonNumber','NucInCodon']]

In [7]:
merge_df = merge_df.drop_duplicates()

In [8]:
merge_df = merge_df.rename(columns = {'Pos':'pos'})

In [9]:
mut_df = mut_df.merge(merge_df, how='left', on=['pos'])

In [10]:
mut_df.head()

,mut_id,pos,parent_nucl,child_nucl,parent_nucl_context,child_nucl_context,parent_node,child_node,topology_dist,GenName,CodonNumber,NucInCodon
0,0,1087,G,A,gtGtt,gtAtt,#54520#,#54519#,0,ORF1ab,274.0,3
1,1,3677,T,C,atTtt,atCtt,#54520#,#54519#,0,ORF1ab,1138.0,1
2,2,7119,C,T,ttCta,ttTta,#54520#,#54519#,0,ORF1ab,2285.0,2
3,3,9565,C,T,ttCtt,ttTtt,#54520#,#54519#,0,ORF1ab,3100.0,3
4,4,10323,A,G,taAgg,taGgg,#54520#,#54519#,0,ORF1ab,3353.0,2


In [11]:
mut_df['RefCodon'] = ''
mut_df['AltCodon'] = ''
mut_df.loc[(mut_df['GenName'].isin(['ORF1ab', 'S', 'ORF8', 'M', 'ORF10', 'ORF7a',
       'ORF3a', 'ORF7b', 'N', 'ORF6',
       'E', 'ORF7a-ORF7b_overlay'])) & (mut_df['NucInCodon'] == '1'),'RefCodon'] = mut_df['parent_nucl'] + mut_df['parent_nucl_context'].astype(str).str[3] + mut_df['parent_nucl_context'].astype(str).str[4]
mut_df.loc[(mut_df['GenName'].isin(['ORF1ab', 'S', 'ORF8', 'M', 'ORF10', 'ORF7a',
       'ORF3a', 'ORF7b', 'N', 'ORF6',
       'E', 'ORF7a-ORF7b_overlay'])) & (mut_df['NucInCodon'] == '2'),'RefCodon'] = mut_df['parent_nucl_context'].astype(str).str[1] + mut_df['parent_nucl'] + mut_df['parent_nucl_context'].astype(str).str[3]
mut_df.loc[(mut_df['GenName'].isin(['ORF1ab', 'S', 'ORF8', 'M', 'ORF10', 'ORF7a',
       'ORF3a', 'ORF7b', 'N', 'ORF6',
       'E', 'ORF7a-ORF7b_overlay'])) & (mut_df['NucInCodon'] == '3'),'RefCodon'] = mut_df['parent_nucl_context'].astype(str).str[0] + mut_df['parent_nucl_context'].astype(str).str[1] + mut_df['parent_nucl']

In [12]:
mut_df.loc[(mut_df['GenName'].isin(['ORF1ab', 'S', 'ORF8', 'M', 'ORF10', 'ORF7a',
       'ORF3a', 'ORF7b', 'N', 'ORF6',
       'E', 'ORF7a-ORF7b_overlay'])) & (mut_df['NucInCodon'] == '1'),'AltCodon'] = mut_df['child_nucl'] + mut_df['child_nucl_context'].astype(str).str[3] + mut_df['child_nucl_context'].astype(str).str[4]
mut_df.loc[(mut_df['GenName'].isin(['ORF1ab', 'S', 'ORF8', 'M', 'ORF10', 'ORF7a',
       'ORF3a', 'ORF7b', 'N', 'ORF6',
       'E', 'ORF7a-ORF7b_overlay'])) & (mut_df['NucInCodon'] == '2'),'AltCodon'] = mut_df['child_nucl_context'].astype(str).str[1] + mut_df['child_nucl'] + mut_df['child_nucl_context'].astype(str).str[3]
mut_df.loc[(mut_df['GenName'].isin(['ORF1ab', 'S', 'ORF8', 'M', 'ORF10', 'ORF7a',
       'ORF3a', 'ORF7b', 'N', 'ORF6',
       'E', 'ORF7a-ORF7b_overlay'])) & (mut_df['NucInCodon'] == '3'),'AltCodon'] = mut_df['child_nucl_context'].astype(str).str[0] + mut_df['child_nucl_context'].astype(str).str[1] + mut_df['child_nucl']

In [13]:
mut_df['NeighL'] = mut_df['parent_nucl_context'].astype(str).str[1]
mut_df['NeighR'] = mut_df['parent_nucl_context'].astype(str).str[3]

mut_df['RefAa'] = ''
mut_df['AltAa'] = ''

In [14]:
mut_df.loc[mut_df['RefCodon']!="","RefAa"] = [seq3(Seq(x).translate()) for x in mut_df.loc[mut_df['RefCodon']!="","RefCodon"].tolist()]

mut_df.loc[mut_df['AltCodon']!="","AltAa"] = [seq3(Seq(x).translate()) for x in mut_df.loc[mut_df['AltCodon']!="","AltCodon"].tolist()]

In [15]:
mut_df['AaSub'] = ""
mut_df['RefCodon'] = mut_df['RefCodon'].astype('str').str.upper()
mut_df['AltCodon'] = mut_df['AltCodon'].astype('str').str.upper()
mut_df['NeighL'] = mut_df['NeighL'].astype('str').str.upper()
mut_df['NeighR'] = mut_df['NeighR'].astype('str').str.upper()
FFcodonsU = ['GCU', 'GCC', 'GCA', 'GCG', 'CGU', 'CGC', 'CGA', 'CGG', 'GGU', 'GGC', 'GGA', 'GGG', 'CUU', 'CUC', 'CUA',
            'CUG', 'CCU', 'CCC', 'CCA', 'CCG', 'UCU', 'UCC', 'UCA', 'UCG', 'ACU', 'ACC', 'ACA', 'ACG', 'GUU', 'GUC',
             'GUA', 'GUG']
FFcononsT = [string.replace("U", "T",3) for string in FFcodonsU]
mut_df.loc[(mut_df['RefAa']==mut_df['AltAa']) &( mut_df['RefAa']!=""),'AaSub'] = 'S'
mut_df.loc[mut_df['RefAa']!=mut_df['AltAa'],'AaSub'] = 'NS'

mut_df.loc[(mut_df['RefCodon'].isin(FFcononsT)) & (mut_df['NucInCodon'] == '3') & (mut_df['RefAa']!=""),'AaSub'] = 'FF'

In [16]:
mut_df['parent_nucl'] = mut_df['parent_nucl'].astype('str').str.replace('T', 'U')
mut_df['child_nucl'] = mut_df['child_nucl'].astype('str').str.replace('T', 'U')
mut_df['RefCodon'] = mut_df['RefCodon'].astype('str').str.replace('T', 'U', 3)
mut_df['AltCodon'] = mut_df['AltCodon'].astype('str').str.replace('T', 'U', 3)
mut_df['NeighL'] = mut_df['NeighL'].astype('str').str.replace('T', 'U')
mut_df['NeighR'] = mut_df['NeighL'].astype('str').str.replace('T', 'U')

In [17]:
mut_df.to_csv('../data/u_mutation_dists.filtered.csv')